### import packages

In [1]:
import heapq
import pickle
import math
import pandas as pd
import numpy as np
import random
import time

from collections import defaultdict

kg_folder = "/Users/yudu/PycharmProjects/PHD_Yu/ExpRec/kge_demo_v5/"

### construction of the input set of annotations
Each line in the given file contains a RDF triple of <movie, property_name, property_value>. For our example, we will extract all the nodes (property values) of the property "dct:subject", which correspond to movie categories.

In [2]:
all_direct_annotations = set()
with open(kg_folder + 'category_tree/ickg_v5_leaf_nodes.nt', 'r') as file:
    for line in file.readlines():
        ppt = line.split('\t')[1]
        item = line.split('\t')[0]
        if "<Item_" in item  and ppt == "<http://purl.org/dc/terms/subject>" and "http" in line.split('\t')[2]:
            all_direct_annotations.add(line.split('\t')[2].strip("\n"))
            
print("The given input set contains {} annotations relevant to movie categories"
      .format(len(all_direct_annotations)))

The given input set contains 22720 annotations relevant to movie categories


### read the file of the entire DBpedia category graph and parse the graph

In [3]:
def read_broader_graph(file_broader):
    """
    input: the entire knowledge graph
    outputs: - nodes (all the nodes in the graph)
             - node_parent_dict (dictionary of "node -> set of parent nodes")
             - node_children_dict (dictionary of "node -> set of children nodes")
    """
    nodes = set()
    node_parent_dict = defaultdict(set)
    node_children_dict = defaultdict(set)
    with open(kg_folder + file_broader, 'r') as file:
        for line in file.readlines():
            if line.split(" ")[1] == "<http://www.w3.org/2004/02/skos/core#broader>":
                node_fils, node_parent = line.split(" ")[0], line.split(" ")[2]
                node_parent_dict[node_fils].add(node_parent)
                node_children_dict[node_parent].add(node_fils)
                nodes.add(node_fils)
                nodes.add(node_parent)
    return node_parent_dict, node_children_dict, nodes

In [4]:
node_parent_dict, node_children_dict, nodes = read_broader_graph('category_tree/categories_skos.ttl')

### find root nodes of the graph, i.e. categories that do not contain parents.

In [5]:
"""
extract all the nodes that do not have parents in the category tree, i.e. the root nodes
"""
def get_roots(node_parent_dict, nodes):
    roots = set()
    nodes_with_parents = node_parent_dict.keys()
    for n in nodes:
        if not n in nodes_with_parents:
            roots.add(n)
    return roots

roots = get_roots(node_parent_dict, nodes)

print("The initial DBpedia category graph contains {} root nodes.".format(len(roots)))

The initial DBpedia category graph contains 61017 root nodes.


In [6]:
# list some of these root nodes for example
list(roots)[:5]

['<http://dbpedia.org/resource/Category:Companies_disestablished_in_1958>',
 '<http://dbpedia.org/resource/Category:1662_disestablishments_in_Taiwan>',
 '<http://dbpedia.org/resource/Category:Business_services_companies_disestablished_in>',
 '<http://dbpedia.org/resource/Category:Philippines–South_Korea_relations>',
 '<http://dbpedia.org/resource/Category:1837_introductions>']

#### add the OWL:Thing root node to the graph structure
A typically ontology structure contains one single root node that generalize all other nodes, for the DBpedia knowledge graph, this root node may not be explicitly set. Here we manually add one external node called "THING", which will be used to generalize the actual roots of the category graph.

In [7]:
nodes.add("<http://dbpedia.org/resource/Category:OWL_THING>")

def add_thing_root(node_parent_dict, node_children_dict):
    owl_thing_id = "<http://dbpedia.org/resource/Category:OWL_THING>"
    for root in roots:
        node_parent_dict[root].add(owl_thing_id)
        node_children_dict[owl_thing_id].add(root)
add_thing_root(node_parent_dict, node_children_dict)

In [8]:
# Now get parents of the '1837_introductions' category, which was considered as one of the root nodes
node_parent_dict['<http://dbpedia.org/resource/Category:1837_introductions>']

{'<http://dbpedia.org/resource/Category:OWL_THING>'}

In [11]:
# Call again the get_roots function
root_node = get_roots(node_parent_dict, nodes)
print(root_node)

{'<http://dbpedia.org/resource/Category:OWL_THING>'}


#### parse the entire graph from leaf-to-root: a post-order parse

In [12]:
"""
algorithm of post order graph parsing. 
Return: 
    - defaultdict(list): root -> [all the descendents by post-order], i.e. from the most specific nodes to the most general ones.
"""

visited = set()
def postOrder(node_children_dict):
    visited.clear()
    root_to_post = defaultdict(list)
    for root in root_node:
        root_to_post[root] = list()
        postOrderRec(root_to_post[root], root, node_children_dict)
    return root_to_post

def postOrderRec(post_order_list, current_node, node_children_dict):
    visited.add(current_node)
    for node in node_children_dict[current_node]:
        if(node not in visited):
            postOrderRec(post_order_list, node, node_children_dict)
    post_order_list.append(current_node)

root2post = postOrder(node_children_dict)

#### remove cycles in the graph
In case where the DBpedia category graph (or in general the entire knowledge graph) may contains cycles, we would like first to roughly remove these cycles.

In [15]:
def remove_cycle_brute(node_children_dict):
    visited_children=set()
    node_children_dict_OK = defaultdict(list)
    node_parent_dict_OK = defaultdict(list)
    
    nb_removed = 0
    nb_remain = 0
    for root in root2post.keys():
        for node in root2post[root]:
            for child in node_children_dict[node]:
                if child in visited_children:
                    nb_remain += 1
                    node_children_dict_OK[node].append(child)
                    node_parent_dict_OK[child].append(node)
                else:
                    nb_removed += 1
            visited_children.add(node)
    print("number of nodes removed: ", nb_removed)
    print("number of nodes remain: ", nb_remain)
    return node_children_dict_OK, node_parent_dict_OK

In [16]:
node_children_dict_OK, node_parent_dict_OK = remove_cycle_brute(node_children_dict)

number of nodes removed:  3661
number of nodes remain:  4141860


In [17]:
# post order parse of the updated graph after removing cycles
root2post = postOrder(node_children_dict_OK)

In [18]:
node_children_dict_OK, node_parent_dict_OK = remove_cycle_brute(node_children_dict_OK)

number of nodes removed:  0
number of nodes remain:  4141860


#### extract-sub ontology for relevant properties
From the output of the previous cell, we see that the graph now contains no cycles, now we will extract from this graph which contains 4141860 nodes, a sub-ontology or a sub-graph, given a set of movie annoations: "all_direct_annotations" that we constructed at the beginning

In [19]:
def sub_onto_nodes(postorderNodes, nodeChildrenDic, all_direct_annotations):
    """
    For purpose of consistency, we employ here the notations that we used in Algorithm 4 of our original paper.
    """    
    SD = defaultdict(set)
    Sal = set()
    for ni, n in enumerate(postorderNodes):
        # n=postorderNodes[ni]
        SD[ni] = set()
        maxSD = 0
        for child in nodeChildrenDic[n]:
            SD[ni] = SD[ni].union(SD[child])
            maxSD = max(maxSD, len(SD[child]))
        if n in all_direct_annotations or len(SD[ni]) > maxSD:
            SD[n].add(n)
            Sal.add(n)
    return Sal

def sub_ontology(postorderNodes, nodeParentDic, nodeChildrenDic, all_direct_annotations):
    """
    For purpose of consistency, we employ here the notations that we used in Algorithm 5 of our original paper.
    """        
    newNodeParentDic = defaultdict(set)
    newNodeChildrenDic = defaultdict(set)
    VR = sub_onto_nodes(postorderNodes, nodeChildrenDic, all_direct_annotations)
    VRRA = defaultdict(set)
    postorderNodes.reverse()
    for ui, u in enumerate(postorderNodes):
        for f in nodeParentDic[u]:
            if not f in VR:
                VRRA[u] = VRRA[u].union(VRRA[f])
            else:
                VRRA[u].add(f)
        if u in VR:
#             print(u)
    #         newPostOrderNodes.add(u)
            for v in VRRA[u]:
                newNodeParentDic[u].add(v)
                newNodeChildrenDic[v].add(u)
    return newNodeParentDic,newNodeChildrenDic

In [20]:
newNodeParentDic, newNodeChildrenDic = sub_ontology(root2post['<http://dbpedia.org/resource/Category:OWL_THING>'], 
                                                    node_parent_dict_OK, 
                                                    node_children_dict_OK, 
                                                    all_direct_annotations)

Let us now see the number of nodes that having parents in the new sub-ontology

In [21]:
len(newNodeParentDic.keys())

28424

Let us now compare this with the number of nodes that having parents in the original entire ontology

In [22]:
len(node_parent_dict_OK.keys())

1639808

We can see that the sub-ontology extraction process has largely reduced the number of nodes in the graph. Let us now do a post order parse of this new sub-ontology to check if these nodes are relevant to the movie domain.

In [23]:
new_parse = postOrder(newNodeChildrenDic)

In [24]:
new_parse

defaultdict(list,
            {'<http://dbpedia.org/resource/Category:OWL_THING>': ['<http://dbpedia.org/resource/Category:HTC–Highroad>',
              '<http://dbpedia.org/resource/Category:Films_set_in_Shanxi>',
              '<http://dbpedia.org/resource/Category:Far_Cry>',
              '<http://dbpedia.org/resource/Category:Doom_(franchise)>',
              '<http://dbpedia.org/resource/Category:First-person_shooters_by_series>',
              '<http://dbpedia.org/resource/Category:Films_set_in_the_White_House>',
              '<http://dbpedia.org/resource/Category:Films_set_in_Brisbane>',
              '<http://dbpedia.org/resource/Category:Films_set_in_Queensland>',
              '<http://dbpedia.org/resource/Category:Films_scored_by_Nadeem–Shravan>',
              '<http://dbpedia.org/resource/Category:2012_fantasy_films>',
              '<http://dbpedia.org/resource/Category:2012_action_thriller_films>',
              '<http://dbpedia.org/resource/Category:2012_science_fictio